# Generate SSH Keys on [Commjhub](https://commjhub.asc.upenn.edu/) for GitHub 🚀🔑⚡

Etienne P Jacquot - [epj@asc.upenn.edu](mailto:epj@asc.upenn.edu)

_______

## Getting Started

Following instructions here: https://docs.github.com/en/github/authenticating-to-github/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent

We do *NOT* want to have undergraduate students entering passwords in plain text!!! SSH is preferred!


### *EXAMPLE: We run this in *one* bash line, instead of multiple python cells:*

``` bash

ssh-keygen -t ed25519 -C "YOUR_EMAIL@gmail.com" -f $HOME/.ssh/id_rsa -N "" <<< y
eval "$(ssh-agent -s)"
ssh-add $HOME/.ssh/id_rsa -vvv

```

______

### To create your SSH key and add to your SSH-Agent:


- Default directory is for `$HOME/.ssh/id_rsa`


- This defaults to *NO* password with the `-N ""` flag


- We include the `<<< y` to overwrite an existing key. Be careful to not rerun this cell *after* uploading your SSH key to github as it'll overwrite!

In [1]:
!ssh-keygen -t ed25519 -C "jacquot.etienne@gmail.com" -f $HOME/.ssh/id_rsa -N "" <<< y && \
eval "$(ssh-agent -s)" && \
ssh-add $HOME/.ssh/id_rsa

Generating public/private ed25519 key pair.
/Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa already exists.
Overwrite (y/n)? Your identification has been saved in /Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa.
Your public key has been saved in /Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:kIt2DTREzg+uLyk4v2htq2ZPnPG4LxqZgvaNreKHxXM jacquot.etienne@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|     o=          |
|     + o         |
|      B          |
|     o B         |
|  ..o + S        |
|. +==E           |
|o*+=+o           |
|=B==B.           |
|**XX==.          |
+----[SHA256]-----+
Agent pid 20062
Identity added: /Commjhub/jupyterhub/comm318_fall2019/{atnjqt}/.ssh/id_rsa (jacquot.etienne@gmail.com)


_______
### Proceed by copying your public key

- Remember to *NEVER* share your private key `.ssh/id_rsa`!

- You need to copy the full result to your clipboard:

In [2]:
!cat $HOME/.ssh/id_rsa.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIAIE5MMMVIe+UMwFKmnPXexBdBC1IpuKsowf7qC2aZ19 jacquot.etienne@gmail.com


### Upload your public key to Github

Following the instructions here: https://docs.github.com/en/github/authenticating-to-github/adding-a-new-ssh-key-to-your-github-account 

- This is a manual step that you must do on your https://github.com/ login... For example:


<img width='800' src='https://imghost.asc.upenn.edu/documentation/commjhub_ssh-pub-key.png' alt='Screenshot of Github New SSH Public Key configuration' style="vertical-align:middle;margin:0px 50px"/> 


___________

### Confirm access by git cloning a private repository via SSH instead of HTTPS

Once your SSH public key is uploaded to github you should be good to go following instructions here: https://docs.github.com/en/github/authenticating-to-github/testing-your-ssh-connection

- **TODO -->** Not sure how to automate the *YES* response for known host, there is a flag which isn't really recommended but lets you ignore host checking. For simplicity of this demonstration let us assume no man-in-the-middle attack on GitHub... 

In [3]:
# For initial git ssh testing, either add your known hosts file OR use flag to ignore
!ssh -o "StrictHostKeyChecking no" -T git@github.com

Hi atnjqt! You've successfully authenticated, but GitHub does not provide shell access.


In [4]:
# You can always confirm your access know that the known host is added:
!ssh -T git@github.com

Hi atnjqt! You've successfully authenticated, but GitHub does not provide shell access.


___________

## Set your global git configuration on Commjhub:

- You must include your `firstname`, `lastname`, and `email`. These are really just unique identifiers so please set to something meaningful.

- Include the `--replace-all` if you want to overwrite all your git configurations. 


### Your personal information here:

In [5]:
firstname = "Etienne"
lastname = "Jacquot"
email = 'jacquot.etienne@gmail.com'

### Set your Git Global configurations:

In [8]:
my_name = '"{} {}"'.format(firstname,lastname) # <--- formatting name

!git config --global --replace-all user.name $my_name

!git config --global --replace-all user.email $email

!git config --global push.default simple # <--- for older versions of git

### Confirm your git configurations:

In [9]:
!git config --list

user.email=jacquot.etienne@gmail.com
user.name=Etienne Jacquot
push.default=simple


_______________

## SSH Clone a private git repo to make changes:

I have an old private git repo here: https://github.com/atnjqt/ASC_Cryptominer

- For this to work, you need *your own private git repository!*

In [12]:
!git clone git@github.com:atnjqt/ASC_Cryptominer.git

Cloning into 'ASC_Cryptominer'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 46 (delta 17), reused 10 (delta 2), pack-reused 0
Receiving objects: 100% (46/46), 3.61 MiB | 0 bytes/s, done.
Resolving deltas: 100% (17/17), done.


__________

## Manually go into your git repo, make a change, then run the below cells to add your commit & push

In my case my git repo is of course: [ASC_Cryptominer/](./ASC_Cryptominer/)


- **Please note:** we include `!cd your_private_repo_name/` at the beginning of each cmd. This is just to *change directory* so our git commands are referencing the correct directory

### Check the status to confirm your changes:

Should have something like

```
# On branch master
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   THE_FILE_THAT_YOU_CHANGED.txt
#
no changes added to commit (use "git add" and/or "git commit -a")

```

In [14]:
!cd ASC_Cryptominer/ && \
git status

# On branch master
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   COMMJHUB_SSH_TEST/TESTING.txt
#
no changes added to commit (use "git add" and/or "git commit -a")


### Add and Commit your changes for the private git repo

- Always include a meaningful message!

In [15]:
!cd ASC_Cryptominer && \
git add --all && \
git commit -m 'testing file changes for private repo ssh'

[master c55fbfe] testing file changes for private repo ssh
 1 file changed, 3 insertions(+), 1 deletion(-)


### Finally, push your changes to a private git repository:


In [17]:
!cd ASC_Cryptominer/ && \
git push -v

Pushing to git@github.com:atnjqt/ASC_Cryptominer.git
Counting objects: 7, done.
Delta compression using up to 16 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 390 bytes | 0 bytes/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To git@github.com:atnjqt/ASC_Cryptominer.git
   c144e44..c55fbfe  master -> master
updating local tracking ref 'refs/remotes/origin/master'


### If that git push fails...


- If you are using branches or get an error, try `git push origin master`


- If you get a warning about `github push.default is unset behavior`... just run (included above so you shouldn't get this, probably git needs to be upgraded on Commjhub VM):

``` bash
git config --global push.default simple
```

______

- If you are still having issues, send me an email at [epj@asc.upenn.edu](mailto:epj@asc.upenn.edu) 🚀

<img width='550' src='https://imghost.asc.upenn.edu/gifs/atn_zoomba.gif'/> 